In [22]:
import numpy as np
import pandas as pd
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\edouc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
#importing car dataset downloaded from "https://www.kaggle.com/CooperUnion/cardataset?select=data.csv"
dataset = pd.read_csv('C:/Users/edouc/Desktop/py/data.csv')
dataset

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11909,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,46120
11910,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,56670
11911,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,50620
11912,Acura,ZDX,2013,premium unleaded (recommended),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,50920


In [24]:
X = dataset.loc[:,'Model':'MSRP'].values.tolist() #storing information to be used to make predictions
y = dataset.loc[:,'Make'].values.tolist() #Brand information to be obtained as a result of the prediction


In [25]:
#Preprocess of the data

documents = []

lemmatizer = WordNetLemmatizer()

for item in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[item]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [lemmatizer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)
    

In [26]:
#vectorize data with with count vectorizer and tf-idf to find numeric values corresponding to text data

vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

In [27]:
#dividing the data to training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [28]:
#training classification model with linear classifier with SGD training.
classifier = SGDClassifier()
classifier.fit(X_train, y_train) 

SGDClassifier()

In [31]:
#Prediction with some data
#For example let's try on; 
# Make    Model  Year  Engine Fuel Type  Engine HP  Engine Cylinders  Transmission Type  Driven_Wheels 
# Nissan  350Z   2009  regular unleaded  306        6                 MANUAL             rear wheel drive 

# Number of Doors  Market Category   Vehicle Size  Vehicle Style  highway MPG  city mpg  Popularity  MSRP
# 2                High-Performance  Compact       Convertible    24           17        2009        36870 

# Let's check whether we will get Nissan as a result by giving the information;
# "306 hp compact regular unleaded 6 engine cylinders"

prediction = "306 hp compact regular unleaded 6 engine cylinders"
prediction = vectorizer.transform([prediction]).toarray()
prediction = tfidfconverter.transform(prediction).toarray()
result = classifier.predict(prediction)[0]

result

'Nissan'